<a href="https://colab.research.google.com/github/ThaniNikolaou/DataJour/blob/main/HomeWork_Week10_Scraping_NikolaouTheodora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [64]:
main_url = "https://www.newsbeast.gr/"
start_url = "https://www.newsbeast.gr/politiki/ekloges-2023/page/"
start_page = 1
end_page = 3

In [65]:
response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [70]:
teaser_articles_list = doc.find('main', {'jsx-4270303143 mainWrap'}).find_all('article')

In [ ]:
teaser_articles_list[0]

In [ ]:
print(teaser_articles_list[0].prettify())

In [75]:
teasers_list = []

for page_num in range(start_page, end_page+1):
    page_url = start_url + str(page_num)
    response = requests.get(page_url)
    doc = BeautifulSoup(response.text, 'html.parser')
    teaser_articles_list = doc.find('main', {'jsx-4270303143 mainWrap'}).find_all('article')

    for teaser in teaser_articles_list:
            story_dict = {}
            try:
                link = teaser.find('a')['href']
                if link.startswith("http"):
                    story_dict['url'] = link
                else:
                    story_dict['url'] = main_url + link
            except:
                story_dict['url'] = ''
                pass
            teasers_list.append(story_dict)

    time.sleep(1)

newsbeast_ekloges_df = pd.DataFrame(teasers_list)


In [ ]:
newsbeast_ekloges_df

In [77]:
article_url = "https://www.newsbeast.gr/politiki/arthro/9803096/ekloges-2023-ti-vouleftiki-edra-tou-voreiou-tomea-krata-o-dimitris-koutsoubas"

In [78]:
full_articles_list = []
for article_url in newsbeast_ekloges_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  article = doc.find('section', {'class': 'article__main'})

  full_article_dict = {}

  full_article_dict['site'] = "https://www.newsbeast.gr/"


  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('div', {'class': 'jsx-1366697334 categories info'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = doc.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = doc.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass


  # full_text
  try:
    p_texts_list = []
    paragraphs = doc.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [ ]:
newsbeast_ekloges_df = pd.DataFrame(full_articles_list)
newsbeast_ekloges_df

In [83]:
newsbeast_ekloges_df.drop_duplicates(inplace=True)

In [87]:
newsbeast_ekloges_df

,site,url,section,title,date,full_text
0,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9805...,ΠΟΛΙΤΙΚΗ,Ξανά εκλογές το φθινόπωρο: Πότε θα γίνουν οι δ...,2023-06-27T11:49:00Z,Μετά τις εθνικές εκλογές της 25ης Ιουνίου ο χρ...
2,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9805...,ΠΟΛΙΤΙΚΗ,Καιρίδης: Δεν θα αφήσουμε το μεταναστευτικό στ...,2023-06-27T11:44:00Z,Για τις προτεραιότητες της κυβέρνησης μίλησε τ...
4,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/ekloges-202...,Εκλογές 2023: Ειδήσεις,Εκλογές 2023: Εκλογές 2023: Ειδήσεις,2023-06-27T11:09:00Z,Στους λόγους για τους οποίους αποχώρησε από το...
6,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9805...,ΠΟΛΙΤΙΚΗ,Η συγκινητική ανάρτηση της Μαρέβα Γκραμπόβσκι ...,2023-06-27T10:52:00Z,«Συγχαρητήρια Κυριάκο! Τα κατάφερες!» Με τα λό...
8,https://www.newsbeast.gr/,https://www.newsbeast.gr//lifestyle/arthro/980...,LIFESTYLE,Δομινίκη Παπαδοπούλου για «Σπαρτιάτες»: «Ήθελα...,2023-06-27T09:52:27Z,Το μοντέλο Δομινίκη Παπαδοπούλου είχε κατέβει ...
...,...,...,...,...,...,...
290,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9799...,ΠΟΛΙΤΙΚΗ,Εκλογές 2023: «Η Κρήτη θα γίνει σύντομα πράσιν...,2023-06-25T18:20:00Z,"Ο πατέρας του Νίκου Ανδρουλάκη, Μαρίνος, με έν..."
292,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9799...,ΠΟΛΙΤΙΚΗ,Μητσοτάκης: Είμαι πρωθυπουργός όλων των Ελλήνω...,2023-06-25T18:17:49Z,Το πρώτο του μήνυμα μετά τη σαρωτική νίκη της ...
294,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9799...,ΠΟΛΙΤΙΚΗ,Εκλογές 2023: Επική απάντηση Χατζηδάκη για Σπα...,2023-06-25T18:15:34Z,"Το κόμμα Σπαρτιάτες μπαίνει στη Βουλή, όπως όλ..."
296,https://www.newsbeast.gr/,https://www.newsbeast.gr//politiki/arthro/9799...,ΠΟΛΙΤΙΚΗ,Για δεύτερη φορά μέσα σε έναν μήνα οι πολίτες ...,2023-06-25T18:12:41Z,Προσερχόμενος σήμερα το πρωί στο Παλαιό Φάληρο...


In [92]:
newsbeast_ekloges_df.to_csv('/content/gdrive/MyDrive/DATA_JOUR/newsbeast_ekloges.csv', index=False)